# AI News Digest Agent

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Define Web Search Tool

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for today's most important AI news for an AI Engineer"""

    return tavily_client.search(
        query=query,
        topics='news',
        search_depth='advanced',
        max_results=10,
        time_range='day',
        include_raw_content=True
    )

In [ ]:
web_search.invoke("Please search for today's most important AI news for an AI Engineer")

## Choose the Model

In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="deepseek-chat")

In [5]:
response = model.invoke("What's an AI agent?")

print(response.content)

An **AI agent** is a software program that can perceive its environment, make decisions, and take actions to achieve specific goals autonomously. Think of it as an intelligent assistant that doesn't just respond to commands but can proactively plan and execute tasks.

### Key Characteristics:
1. **Autonomy** – Operates without constant human intervention.
2. **Perception** – Uses sensors, data inputs, or APIs to gather information.
3. **Decision-making** – Employs reasoning, rules, or machine learning models to choose actions.
4. **Action** – Interacts with environments (digital or physical) via APIs, tools, or robotic actuators.
5. **Goal-oriented** – Works toward defined objectives, sometimes learning and adapting over time.

### Common Examples:
- **Virtual Assistants** (e.g., Siri, Alexa) – Basic agents that follow commands.
- **Customer Service Chatbots** – Handle queries and escalate issues.
- **Autonomous Vehicles** – Perceive roads, navigate, and avoid obstacles.
- **Trading Bo

## Defining Response Format

In [6]:
from pydantic import BaseModel, Field
from typing import List

class AINews(BaseModel):
    title: str
    url: str
    explanation: str = Field(
        description="A detailed explanation (10 sentences minimum) covering: What happened, Why it's significant, How it impacts AI engineers or the industry"
    ) 

class AINewsDigest(BaseModel):
    news_items: List[AINews]

## Create the Agent

In [7]:
system_prompt = """
You are an AI News Assistant. Your job is to search for and explain the latest AI technology news.

When users ask about AI news:
1. Use the web_search tool to find recent AI news articles from the last 24 hours
2. Choose the 5 most important news items for an AI Engineer
3. Provide a detailed explanation (10 sentences minimum) for each news item that covers:
   - What happened
   - Why it's significant
   - How it impacts AI engineers or the industry

Be conversational and helpful.
"""

In [8]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="deepseek-chat",
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver(),
    response_format=AINewsDigest,
)

In [9]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "0"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="Is there new exciting news?")]},
    config
)

In [10]:
pprint(response['structured_response'].news_items)

Pretty printing has been turned OFF


## Check the Conversation History

In [ ]:
from pprint import pprint

# Get the current/latest checkpoint
checkpoint = agent.checkpointer.get(config)

pprint(checkpoint['channel_values']['messages'])

## Format the Response For the User

In [12]:
for item in response['structured_response'].news_items:
    print(f"Title: {item.title}\nURL: {item.url}\nExplanation: {item.explanation}\n\n")

Title: 3D Chip Breakthrough Claims to Solve AI's Memory Wall Problem
URL: https://aijungle.substack.com/p/ai-stars-of-the-week-newsletter-december-1ef
Explanation: US researchers have unveiled a monolithic 3D chip architecture that claims to eliminate the memory wall bottleneck constraining AI performance, potentially enabling hardware that's 1,000 times faster than current systems. The memory wall refers to the fundamental mismatch between processor speed and memory access time where modern AI chips spend most of their time waiting for data to move between memory and processing units. The new 3D architecture addresses this by vertically stacking processing and memory layers with direct connections, dramatically reducing the physical distance data must travel. This breakthrough is significant because the data transfer bottleneck has become the primary constraint limiting AI system performance rather than raw computational power. For AI engineers, this means future hardware could dramat